In [ ]:
import os
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from pathlib import Path
from turmoric.apply_thresholds import apply_li_threshold
from skimage import filters
from skimage.segmentation import clear_border
from skimage.morphology import remove_small_objects
from skimage.measure import block_reduce, label, regionprops
from scipy import ndimage

In [ ]:
def create_microglia_mask(image, threshold_method=filters.threshold_li):

    thresh_li = threshold_method(image)
    binary_li = image > thresh_li

    objects = label(binary_li)
    objects = clear_border(objects)
    large_objects = remove_small_objects(objects, min_size=50000)
    small_objects = label((objects ^ large_objects) > thresh_li)

    binary_li = ndimage.binary_fill_holes(remove_small_objects(small_objects > thresh_li, min_size=500))

    #scaled_img = ((image - image.min()) * (1/(image.max() - image.min()) * 255)).astype('uint8')
    #hist = np.histogram(scaled_img.flatten(), range=[0,50], bins=50)
    return binary_li

In [ ]:
def get_object_sizes(image, threshold_method=filters.threshold_li):

    thresh_li = threshold_method(image)
    binary_li = image > thresh_li
    binary_li = ndimage.binary_fill_holes(remove_small_objects(binary_li, min_size=500))

    objects = label(binary_li)
    objects = clear_border(objects)
    
    props = regionprops(objects)

    # Get sizes for each label
    label_sizes = [prop.area for prop in props]
    return label_sizes

In [ ]:
files = Path("/Users/nelsschimek/Documents/nancelab/Data/tommy_data/Sham_3/Left_HC")


In [ ]:
npy_files = list(files.glob("*.tif"))

# If you want full paths as strings:
npy_file_paths = [str(f) for f in npy_files]
(npy_file_paths)

In [ ]:
microglia_images = [tiff.imread(f) for f in sorted(npy_file_paths)]
print(len(microglia_images))

In [ ]:
mean_masks = [create_microglia_mask(f, threshold_method=filters.threshold_mean) for f in microglia_images]
li_masks = [create_microglia_mask(f, threshold_method=filters.threshold_li) for f in microglia_images]


In [ ]:
sizes = get_object_sizes(microglia_images[1])
min(sizes)

In [ ]:
plt.hist((sizes))

In [ ]:
columns = 3
rows = len(li_masks)

fig, axes = plt.subplots(len(li_masks), columns, figsize=(10*columns, rows*10))
for ax, li, mean, raw in zip(axes, li_masks, mean_masks, microglia_images):
    ax[0].imshow(raw > filters.threshold_li(raw), cmap="gray")
    ax[1].imshow(mean, cmap="gray")
    ax[2].imshow(li, cmap="gray")

In [ ]:
npy_files = Path("/Users/nelsschimek/Documents/nancelab/Data/tommy_data/Sham_3/li_thresh/Left_HC")


test_masks = npy_files = list(npy_files.glob("*.npy"))

# If you want full paths as strings:
npy_file_paths = [str(f) for f in npy_files]
(npy_file_paths)

In [ ]:
loaded_masks = [np.load(f) for f in npy_file_paths]

In [ ]:
loaded_masks[0]

In [ ]:
columns = 1
rows = len(li_masks)

fig, axes = plt.subplots(len(loaded_masks), columns, figsize=(10*columns, rows*10))
for ax, li in zip(axes, loaded_masks):
    ax.imshow(li)
   